In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np 

In [2]:
#Load the trained model, encoder, scaler, onehot pickle
model = load_model('model.h5')

#loading the encoder and scaler
with open('one_hot_encoder_geo.pkl','rb') as file:
    ohe_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [8]:
#Input Data Example
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

df_ex = pd.DataFrame([input_data])
df_ex.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [15]:
geo_encoded = ohe_encoder_geo.transform([df_ex['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = ohe_encoder_geo.get_feature_names_out())
geo_encoded_df.head()

e:\COLLEGE\GEN -AI - LLM\NLP FOR ML\Projects\ANN - Churn Modelling Dataset\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [16]:
df_ex = pd.concat([df_ex,geo_encoded_df],axis =1)
df_ex.drop('Geography',axis = 1,inplace = True)
df_ex.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [18]:
df_ex['Gender'] = label_encoder_gender.transform(df_ex['Gender'])
df_ex.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [19]:
df_ex_ss = scaler.transform(df_ex)
prediction = model.predict(df_ex_ss)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step


array([[0.03915046]], dtype=float32)

In [20]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.039150458)

In [21]:
if prediction_proba>0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
